In [ ]:
import random
import torch
import sys
import numpy as np
import math
import matplotlib.pyplot as plt
from timeit import default_timer
from scipy.io import loadmat
import yaml
import gc
from pprint import pprint

sys.path.append("../")


torch.set_printoptions(precision=16)


torch.manual_seed(0)
np.random.seed(0)

data_path = "..\data\car\car_data.npz"
data = np.load(data_path)


all_points = data["points"]
all_triangles = data["triangles"]
all_normals = data["normals"]
all_pressures = data["pressures"]


print(f"Points: {all_points.shape}")
print(f"Triangles: {all_triangles.shape}")
print(f"Normals: {all_normals.shape}")
print(f"Pressures: {all_pressures.shape}")

#Points: (611, 3586, 3)
# Triangles: (611, 7168, 3)
# Normals: (611, 3586, 3)
# Pressures: (611, 3682)

Points: (611, 3586, 3)
Triangles: (611, 7168, 3)
Normals: (611, 3586, 3)
Pressures: (611, 3682)


In [4]:
from mayavi import mlab
# 使用Mayavi绘制3D散点图
index = -100+14
plot_vecs = True
x,y,z = all_points[index].transpose()
triangles = all_triangles[0]
src = mlab.triangular_mesh(x,y,z, triangles,
                           representation='wireframe')  # wireframe模式显示网格线

lines = mlab.pipeline.extract_edges(src)
tubes = mlab.pipeline.tube(lines, tube_radius=0.005)  # 调整 tube_radius 来改变宽度
mlab.pipeline.surface(tubes, color=(0, 1, 0))

if plot_vecs:
    u, v, w = all_normals[index].transpose()
    vecs = mlab.quiver3d(x, y, z, u, v, w, mode='arrow', color=(0, 0, 0), scale_factor=0.15)

# 设置视图
mlab.view(azimuth=180, elevation=70, focalpoint=[0, 0, 0],
          distance=10, roll=90)

# 显示图形
mlab.show()

In [3]:
from timeit import default_timer

grid_weight = np.zeros((611,3586))
for index in range(611):
    t1 = default_timer()
    points = all_points[index]
    Triangles = all_triangles[index]
    # 计算每个点的面积贡献
    num_points = points.shape[0]
    

    # 计算每个三角形的面积
    def triangle_area(p1, p2, p3):
        vec1 = p2 - p1
        vec2 = p3 - p1
        cross_product = np.cross(vec1, vec2)
        area = 0.5 * np.linalg.norm(cross_product)
        return area

    # 构造邻接矩阵
    adjacency_matrix = [[] for _ in range(num_points)]
    for tri in Triangles:
        adjacency_matrix[tri[0]].append(tri)
        adjacency_matrix[tri[1]].append(tri)
        adjacency_matrix[tri[2]].append(tri)

    # 计算每个点的面积贡献
    for i in range(num_points):
        total_area = 0.0
        num_triangles = len(adjacency_matrix[i])
        for tri in adjacency_matrix[i]:
            total_area += triangle_area(points[tri[0]], points[tri[1]], points[tri[2]])
        grid_weight[index,i] = total_area / 3
    t2 = default_timer()
    print(f'index = {index}, time: {t2-t1}')
print(grid_weight.shape)
np.save('data/grid_weight.npy',grid_weight)

index = 0, time: 0.5841346000006524
index = 1, time: 0.5877627999998367
index = 2, time: 0.6761764000002586
index = 3, time: 0.6003768999999011
index = 4, time: 0.581922599999416
index = 5, time: 0.5929174000002604
index = 6, time: 0.5877997000006872
index = 7, time: 0.5786425000005693
index = 8, time: 0.5974954999992406
index = 9, time: 0.6030363999998372
index = 10, time: 0.5825143000001844
index = 11, time: 0.5890380999999252
index = 12, time: 0.5809763000006569
index = 13, time: 0.580988000000616
index = 14, time: 0.5895166000000245
index = 15, time: 0.5911128999996436
index = 16, time: 0.6074104999997871
index = 17, time: 0.595057700000325
index = 18, time: 0.5950008999998317
index = 19, time: 0.5832294000001639
index = 20, time: 0.578190999999606
index = 21, time: 0.5677797999996983
index = 22, time: 0.5783987000004345
index = 23, time: 0.5786066999999093
index = 24, time: 0.5697643000003154
index = 25, time: 0.5779940000002171
index = 26, time: 0.5878204000000551
index = 27, tim

In [2]:
from mayavi import mlab

# 使用Mayavi绘制3D散点图
index = 0
m = 20
print(grid_weight[0,:m])


x,y,z = all_points[index].transpose()
special_points = all_points[index,:m,:]
special_x, special_y, special_z = special_points.T
triangles = all_triangles[0]
src = mlab.triangular_mesh(x,y,z, triangles,
                           representation='wireframe')  # wireframe模式显示网格线

mlab.points3d(special_x, special_y, special_z, mode='point', color=(1, 0, 0), scale_factor=1)
for i, point in enumerate(special_points):
    mlab.text3d(point[0], point[1], point[2], str(i + 1), scale=(0.1, 0.1, 0.1))

# 设置视图
mlab.view(azimuth=180, elevation=70, focalpoint=[0, 0, 0],
          distance=10, roll=90)

# 显示图形
mlab.show()

NameError: name 'grid_weight' is not defined

In [14]:
from mayavi import mlab
import imageio
index = 300

mlab.options.offscreen = True  # 设置为无头模式

x,y,z = all_points[index].transpose()
u, v, w = all_normals[index].transpose()
fig1 = mlab.figure(size=(2000,2000), bgcolor=(1, 1, 1))
mlab.view(azimuth=0, elevation=70, focalpoint=[0, 0, 0], distance=10, roll=0, figure=fig1)
mlab.quiver3d(x, y, z, u, v, w, mode='arrow', color=(0, 1, 0), scale_factor=0.075, figure=fig1)


fig2 = mlab.figure(size=(2000,2000), bgcolor=(1, 1, 1))
mlab.view(azimuth=0, elevation=70, focalpoint=[0, 0, 0], distance=10, roll=0, figure=fig2)
mlab.quiver3d(x, y, z, u, v, w, mode='arrow', color=(0, 1, 0), scale_factor=0.075, figure=fig2)

fig3 = mlab.figure(size=(2000,2000), bgcolor=(1, 1, 1))
mlab.view(azimuth=0, elevation=70, focalpoint=[0, 0, 0], distance=10, roll=0, figure=fig3)
mlab.points3d(x, y, z, z, color=(0, 1, 0), mode='sphere', scale_factor=0.075, figure=fig3)
combined_image = np.hstack([
    mlab.screenshot(figure=fig1),
    mlab.screenshot(figure=fig2),
    mlab.screenshot(figure=fig3)
])
filename = 'combined_plots.png'
imageio.imwrite(filename, combined_image)

mlab.close(fig1)
mlab.close(fig2)

In [41]:
t = torch.rand(3,4)
a,b,c = t
print(t)
print(a,b,c)

tensor([[0.0223257541656494, 0.1688589453697205, 0.2938884496688843,
         0.5185217857360840],
        [0.6976675987243652, 0.8000113964080811, 0.1610294580459595,
         0.2822685837745667],
        [0.6816085577011108, 0.9151939749717712, 0.3970999121665955,
         0.8741558790206909]])
tensor([0.0223257541656494, 0.1688589453697205, 0.2938884496688843,
        0.5185217857360840]) tensor([0.6976675987243652, 0.8000113964080811, 0.1610294580459595,
        0.2822685837745667]) tensor([0.6816085577011108, 0.9151939749717712, 0.3970999121665955,
        0.8741558790206909])
